In [17]:
%%writefile HangmanGame.h

// HangmanGame.h
#ifndef HANGMANGAME_H
#define HANGMANGAME_H
#define MAXLENGTH 100
#include <cstring>
#include <string>

const int MAX_ACHIEVEMENTS = 10;

class HangmanGame {
private:
    class Node {
    public:
        char word[MAXLENGTH];
        Node* next;

        Node(const char* w) {
            strcpy(word, w);
            next = nullptr;
        }
    };

    Node* current;
    Node* wordList;
    char unknown[MAXLENGTH];
    char letter;
    int num_of_wrong_guesses;

public:
    HangmanGame();
    ~HangmanGame();
    void run();
    void saveGame();
    void loadGame();
    int getScore() const;
    void updateScore(int points);
    void addAchievement(const std::string& achievement);
    void displayAchievements() const;
    void displayGuessesLeft();

private:
    int score;
    bool isRemainingGuessesShown;
    int numAchievements;
    void addWord(const char* word);
    void initUnknown();
    void displayWelcomeMessage();
    void displayGuessesLeft();
    void displayWordProgress();
    void displayGameOver();
    void letterFill(char guess);
    std::string achievements[MAX_ACHIEVEMENTS];

};

const int MAX_TRIES = 7; // Global constant declaration

#endif  // HANGMANGAME_H





Overwriting HangmanGame.h


In [18]:
%%writefile HangmanGame.cpp
// HangmanGame.cpp
#include "HangmanGame.h"
#include <iostream>
#include <cstring>
#include <ctime>
#include <fstream>
#include <string>

using namespace std;

HangmanGame::HangmanGame() : num_of_wrong_guesses(0), current(nullptr), wordList(nullptr), isRemainingGuessesShown(false) {
    // Add words to the linked list
    addWord("india");
    addWord("vietnam");
    addWord("nepal");
    addWord("japan");
    addWord("fiji");
    addWord("france");
    addWord("iran");
    addWord("brunei");
    addWord("italy");
    addWord("taiwan");
}

HangmanGame::~HangmanGame() {
    // Clean up the linked list
    Node* current = wordList;
    while (current != nullptr) {
        Node* temp = current;
        current = current->next;
        delete temp;
    }
}

void HangmanGame::addWord(const char* word) {
    Node* newNode = new Node(word);

    if (wordList == nullptr) {
        wordList = newNode;
    } else {
        Node* current = wordList;
        while (current->next != nullptr) {
            current = current->next;
        }
        current->next = newNode;
    }
}

int HangmanGame::getScore() const {
    return score;
}

void HangmanGame::updateScore(int points) {
    score += points;
}

void HangmanGame::addAchievement(const std::string& achievement) {
    if (numAchievements < MAX_ACHIEVEMENTS) {
        achievements[numAchievements] = achievement;
        numAchievements++;
    }
}

void HangmanGame::displayAchievements() const {
    std::cout << "Achievements:\n";
    for (int i = 0; i < numAchievements; i++) {
        std::cout << "- " << achievements[i] << "\n";
    }
}

void HangmanGame::run() {
    num_of_wrong_guesses = 0;
    srand(time(0)); // Seed the random number generator

    displayWelcomeMessage();

    // Randomly select a word from the linked list
    Node* temp = wordList;
    int count = 0;
    while (temp != nullptr) {
        count++;
        temp = temp->next;
    }
    int randomIndex = rand() % count;

    temp = wordList;
    for (int i = 0; i < randomIndex; i++) {
        temp = temp->next;
    }

    current = temp; // Update the current pointer

    strcpy(unknown, current->word);

    initUnknown(); // Initialize the unknown array

    // Game loop
    while (num_of_wrong_guesses < MAX_TRIES) {
        displayWordProgress();
        cout << "\n\nGuess a letter (enter 'Q' to quit): ";
        cin >> letter;

        if (letter == 'Q' || letter == 'q') {
            cout << "\nQuitting the game...\n";
            return;
        }

        letterFill(letter);

        if (strcmp(current->word, unknown) == 0) {
            displayWordProgress();
            cout << " Yeah! You got it!";

            int points = MAX_TRIES - num_of_wrong_guesses;
            updateScore(points);

            if (points >= 5) {
                addAchievement("Master Guesser");
            }

            return;
        }

        displayGuessesLeft();
    }

    displayGameOver();
}

void HangmanGame::letterFill(char guess) {
    int i;
    bool correctGuess = false;
    for (i = 0; current->word[i] != '\0'; i++) {
        if (guess == current->word[i]) {
            unknown[i] = guess;
            correctGuess = true;
        }
    }

    if (!correctGuess) {
        num_of_wrong_guesses++;
    }
}


void HangmanGame::initUnknown() {
    int i;
    int length = strlen(current->word);  // Use current pointer
    for (i = 0; i < length; i++)
        unknown[i] = '*';
    unknown[i] = '\0';
}

void HangmanGame::displayWelcomeMessage() {
    cout << "\n\nWelcome to hangman...Guess a country Name";
    cout << "\n\nEach letter is represented by a star.";
    cout << "\n\nYou have to type only one letter in one try";
    cout << "\n\nYou have " << MAX_TRIES << " tries to try and guess the word.";
    cout << "\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~";
}

void HangmanGame::displayGuessesLeft() {
    if (!isRemainingGuessesShown) {
        int remainingGuesses = MAX_TRIES - num_of_wrong_guesses;

        if (remainingGuesses == 1) {
            cout << "You have 1 guess left." << endl;
        } else if (remainingGuesses > 0) {
            cout << "You have " << remainingGuesses << " guesses left." << endl;
        } else {
            cout << "No guesses left." << endl;
        }

        isRemainingGuessesShown = true;
    }
}

void HangmanGame::displayWordProgress() {
    cout << "\n\n" << unknown;
}

void HangmanGame::displayGameOver() {
    cout << "\nSorry, you lose...you've been hanged." << endl;
    cout << "The word was: " << current->word << endl;

    // Display the hangman
    cout << "  _________     " << endl;
    cout << "  |         |   " << endl;
    cout << "  |         O   " << endl;

    if (num_of_wrong_guesses >= 2) {
        cout << "  |        /";
        if (num_of_wrong_guesses >= 3)
            cout << "|";
        cout << endl;
    } else {
        cout << "  |" << endl;
    }

    if (num_of_wrong_guesses >= 4) {
        cout << "  |         |   " << endl;
    } else {
        cout << "  |" << endl;
    }

    if (num_of_wrong_guesses >= 5) {
        cout << "  |        /";
        if (num_of_wrong_guesses >= 6)
            cout << "\\";
        cout << endl;
    } else {
        cout << "  |" << endl;
    }

    cout << " _|_           " << endl;
    cout << "|   |______    " << endl;
    cout << "|__________|   " << endl;
}

void HangmanGame::saveGame() {
    ofstream outFile("hangman_save.txt", ios::out | ios::binary);

    if (!outFile) {
        cout << "Error saving game.\n";
        return;
    }

    // Save game state
    outFile.write(reinterpret_cast<const char*>(&num_of_wrong_guesses), sizeof(num_of_wrong_guesses));
    outFile.write(reinterpret_cast<const char*>(&letter), sizeof(letter));
    outFile.write(reinterpret_cast<const char*>(unknown), sizeof(unknown));
    outFile.write(reinterpret_cast<const char*>(current->word), sizeof(current->word));

    outFile.close();

    cout << "Game saved.\n";
}

void HangmanGame::loadGame() {
    ifstream inFile("hangman_save.txt", ios::in | ios::binary);

    if (!inFile) {
        cout << "Error loading game.\n";
        return;
    }

    // Load game state
    inFile.read(reinterpret_cast<char*>(&num_of_wrong_guesses), sizeof(num_of_wrong_guesses));
    inFile.read(reinterpret_cast<char*>(&letter), sizeof(letter));
    inFile.read(reinterpret_cast<char*>(unknown), sizeof(unknown));
    inFile.read(reinterpret_cast<char*>(current->word), sizeof(current->word));

    inFile.close();

    cout << "Game loaded.\n";

    // Display the loaded game state
    displayGuessesLeft();
    displayWordProgress();
}

void displayMenu() {
    cout << "====================================\n";
    cout << "          Hangman Game Menu          \n";
    cout << "====================================\n";
    cout << "1. Start New Game\n";
    cout << "2. Load Saved Game\n";
    cout << "3. Exit\n";
    cout << "====================================\n";
    cout << "Enter your choice: ";
}

int main() {
    HangmanGame game;

    int choice;
    displayMenu();
    cin >> choice;

    while (choice != 3) {
        if (choice == 1) {
            game.run();
        } else if (choice == 2) {
            game.loadGame();
        } else {
            cout << "Invalid choice. Please try again.\n";
        }

        // After the game finishes, prompt the user to continue or exit
        cout << "\n====================================\n";
        cout << "1. Play Again\n";
        cout << "2. Save Game and Exit\n";
        cout << "3. Exit Without Saving\n";
        cout << "====================================\n";
        cout << "Enter your choice: ";
        cin >> choice;

        if (choice == 1) {
            game.run();
        } else if (choice == 2) {
            game.saveGame();
            cout << "Exiting the game.\n";
            break;
        } else if (choice == 3) {
            cout << "Exiting the game without saving.\n";
            break;
        } else {
            cout << "Invalid choice. Exiting the game.\n";
            break;
        }
    }

    return 0;
}


Overwriting HangmanGame.cpp


In [ ]:
!g++ HangmanGame.cpp -o hgman.elf
!./hgman.elf